In [9]:
#Sytem
import sys,os
import numpy as np
import pandas as pd
import warnings
import time
import random
import IPython
warnings.filterwarnings('ignore')

#Visualidation
import sweetviz as sv
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#EDA
import sweetviz as sv
import sklearn
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



#model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from skopt import BayesSearchCV
from sklearn import model_selection
from xgboost import XGBClassifier



print('-'*40)

print("Python version: {}". format(sys.version))

print("pandas version: {}". format(pd.__version__))

print("matplotlib version: {}". format(matplotlib.__version__))

print("NumPy version: {}". format(np.__version__))


print("scikit-learn version: {}". format(sklearn.__version__))
print('-'*40)


----------------------------------------
Python version: 3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
pandas version: 1.3.2
matplotlib version: 3.4.3
NumPy version: 1.22.4
scikit-learn version: 0.24.2
----------------------------------------


In [2]:
Target = "Perished"
PATH_LOG = "./log.csv"
ID = 00
SEED = 42

In [4]:
train = pd.read_csv("./df/EDA_10_train.csv")
test = pd.read_csv("./df/EDA_10_test.csv")

print('-'*25)
print("Shape of Train Data\n[row :{},column :{}]".format(
    train.shape[0], train.shape[1]))
print('-'*25)
print("Shape of Test Data\n[row :{},column :{}]".format(
    test.shape[0], test.shape[1]))

-------------------------
Shape of Train Data
[row :891,column :12]
-------------------------
Shape of Test Data
[row :418,column :12]


In [5]:
X = train.iloc[:, 2:].values
y = train.iloc[:, 1].values
X_test = test.iloc[:, 2:].values

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#Machine Learning Algorithm (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),

    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),

    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),

    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),

    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),

    #Trees
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),

    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),


    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()
]


#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
# run model 10x with 60/30 split intentionally leaving out 10%
cv_split = model_selection.ShuffleSplit(
    n_splits=10, test_size=.3, train_size=.6, random_state=0)

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Train Accuracy Mean',
               'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD', 'MLA Time']
MLA_compare = pd.DataFrame(columns=MLA_columns)

#create table to compare MLA predictions
MLA_predict = y_train
model_list = {}
score_list =[]
#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(
        alg, X_train, y_train, cv=cv_split, return_train_score=True,return_estimator=True)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index,
                    'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index,
                    'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    # let's know the worst that can happen!
    MLA_compare.loc[row_index,
                    'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3
        
    #save MLA predictions - see section 6 for usage
    alg.fit(X_train, y_train)
    
    score_list.append(str(round(cv_results['train_score'].mean(),4)))
    try:
        model_list[str(round(cv_results['train_score'].mean(),4))]=alg.predict_proba(X_test)
    except:
        pass

    row_index += 1

score_list.sort(reverse=True)
#print and sort table: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sort_values.html
MLA_compare.sort_values(by=['MLA Test Accuracy Mean'],
                        ascending=False, inplace=True)
MLA_compare


,MLA Name,MLA Parameters,MLA Train Accuracy Mean,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
3,GradientBoostingClassifier,"{'ccp_alpha': 0.0, 'criterion': 'friedman_mse'...",0.925995,0.81028,0.059542,0.142711
0,AdaBoostClassifier,"{'algorithm': 'SAMME.R', 'base_estimator': Non...",0.859953,0.799533,0.062521,0.101259
6,LogisticRegressionCV,"{'Cs': 10, 'class_weight': None, 'cv': None, '...",0.815222,0.792056,0.069601,0.706973
17,LinearDiscriminantAnalysis,"{'covariance_estimator': None, 'n_components':...",0.809602,0.787383,0.065828,0.00341
4,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",0.986651,0.785981,0.039052,0.175905
13,NuSVC,"{'break_ties': False, 'cache_size': 200, 'clas...",0.827166,0.782243,0.073968,0.092465
2,ExtraTreesClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",0.986651,0.776168,0.049939,0.198698
1,BaggingClassifier,"{'base_estimator': None, 'bootstrap': True, 'b...",0.9726,0.769159,0.06181,0.040429
19,XGBClassifier,"{'objective': 'binary:logistic', 'use_label_en...",0.983372,0.768224,0.05101,0.091088
9,BernoulliNB,"{'alpha': 1.0, 'binarize': 0.0, 'class_prior':...",0.795316,0.757944,0.050467,0.001308
